In [8]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from dotenv import load_dotenv

load_dotenv()
model_name = "medgemma-4b-it"
emb_model_name = "nomic-embed-text-v1.5"
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    model=model_name,
    api_key="lm-studio"
)

In [11]:
from typing import List
import requests
from langchain.embeddings.base import Embeddings
class NomicEmbeddings(Embeddings):
    def __init__(self, model_name:str, base_url:str="http://localhost:1234/v1", api_key:str="lm-studio"):
        self.model_name = model_name
        self.base_url = base_url
        self.api_key = api_key
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self.embed_query(text) for text in texts]
    
    def embed_query(self, text: str) -> List[float]:
        url = f"{self.base_url}/embeddings"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-type": "application/json"
        }
        payload = {
            "model": self.model_name,
            "input": text
        }

        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()['data'][0]['embedding']
    
embeddings = NomicEmbeddings(model_name="nomic-embed-text-v1.5")

# Load the Sample Document

In [13]:
text_file = "agents_overview.txt"

loader = TextLoader(text_file, encoding="utf-8")
documents = loader.load()
documents

[Document(metadata={'source': 'agents_overview.txt'}, page_content='**All About AI Agents**\n\n---\n\n**1. What Are AI Agents?**\nAI Agents are autonomous systems designed to perceive their environment, make decisions, and take actions to achieve specific goals. Unlike traditional software, AI agents can adapt their behavior based on feedback and changing contexts.\n\n---\n\n**2. Key Characteristics**\n\n* **Autonomy:** Operate independently without human intervention.\n* **Reactivity:** Respond to environmental changes.\n* **Proactiveness:** Take initiative to meet objectives.\n* **Social Ability:** Communicate with humans or other agents.\n\n---\n\n**3. Types of AI Agents**\n\n* **Simple Reflex Agents:** Act only on current percepts.\n* **Model-Based Reflex Agents:** Maintain an internal state.\n* **Goal-Based Agents:** Act to achieve specific goals.\n* **Utility-Based Agents:** Maximize a given utility function.\n* **Learning Agents:** Improve performance over time.\n\n---\n\n**4. A

In [14]:
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
text_splitter

In [15]:
docs = text_splitter.split_documents(documents)
docs

[Document(metadata={'source': 'agents_overview.txt'}, page_content='**All About AI Agents**\n\n---\n\n**1. What Are AI Agents?**\nAI Agents are autonomous systems designed to perceive their environment, make decisions, and take actions to achieve specific goals. Unlike traditional software, AI agents can adapt their behavior based on feedback and changing contexts.\n\n---'),
 Document(metadata={'source': 'agents_overview.txt'}, page_content='---\n\n**2. Key Characteristics**\n\n* **Autonomy:** Operate independently without human intervention.\n* **Reactivity:** Respond to environmental changes.\n* **Proactiveness:** Take initiative to meet objectives.\n* **Social Ability:** Communicate with humans or other agents.\n\n---'),
 Document(metadata={'source': 'agents_overview.txt'}, page_content='---\n\n**3. Types of AI Agents**'),
 Document(metadata={'source': 'agents_overview.txt'}, page_content='* **Simple Reflex Agents:** Act only on current percepts.\n* **Model-Based Reflex Agents:** Ma

# Convert Text to Vectors and Index the FAISS

In [16]:
vectorstore = FAISS.from_documents(docs, embeddings)

# Setup Retrieval Augmented QA Chain

In [23]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2}),
    return_source_documents=True
)
qa_chain

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001A077521B20>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001A0780FDC70>, root_client=<openai.OpenAI object at

# Ask Multiple Questions and Display Answers with Sources

In [25]:
questions = [
    "What are AI Agents",
    "What are evaluation strategies for AI Agents",
    "Components of AI Agents"
]

for q in questions:
    response = qa_chain.invoke({"query": q})
    print("*"*100)
    print(f"Question: {q}")
    print(f"Answer: {response['result']}")
    print(f"Source Document Snippets: ")
    for doc in response["source_documents"]:
        print(f"- {doc.page_content[:150]}...")

****************************************************************************************************
Question: What are AI Agents
Answer: AI Agents are autonomous systems designed to perceive their environment, make decisions, and take actions to achieve specific goals. Unlike traditional software, AI agents can adapt their behavior based on feedback and changing contexts.

Source Document Snippets: 
- **All About AI Agents**

---

**1. What Are AI Agents?**
AI Agents are autonomous systems designed to perceive their environment, make decisions, and ...
- ---

**3. Types of AI Agents**...
****************************************************************************************************
Question: What are evaluation strategies for AI Agents
Answer: Based on the provided context, here are some evaluation strategies for AI agents:

*   Task success metrics
*   G-Eval (LLM-based grading)
*   Human-in-the-loop assessments
*   Simulation environments (e.g., MiniWoB++)

Source Document Snip